In [ ]:
Ngắn gọn lại: Lý do nhận được dữ liệu "sạch" ở Component là do cơ chế "Khớp tên Key"
1. **Luật của RTK Query**:
  - Nó chỉ chấp nhận kết quả trả về từ `baseQuery` nếu nằm trong key có tên là `data` (hoặc `error`)

2. **Dữ liệu của bạn**:
  - Object trả về từ (baseQuery) (do Axios/Backend) tình cờ có sẵn thuộc tính là `data`

3. **Hành động tự động**:
  - RTK Query thấy key `data`
  - Nó *lấy nội dung bên trong* key đó (tức `items`, `pagination`)
  - Nó vứt bỏ các vỏ bọc bên ngoài (như `status`, `headers`, `config`)

# Công thức đơn giản:
baseQuery trả về `{ data: X }` ➔ RTK Query sẽ đưa X cho Component.

In [ ]:
1. API gốc trả về:

```js

  {
    status: 200,
    data: { items: [...], pagination: {...} }
  }

```
-------------------------- 
2. Axios trả về response có cấu trúc:

```js

  {
    data: { items: [...], pagination: {...} },
    status: 200,
    headers: {...}
  }

```
-------------------------- 
3. **RTK Query unwrap**: hook `useGetProductsQuery()` luôn lấy `baseQuery.data` làm payload.
  - Nếu `baseQuery` return `response` (Axios response), hook trả về `response.data` → chỉ còn `{ items, pagination }`
  - Nếu muốn giữ nguyên toàn bộ `response` trong hook, baseQuery phải `return { data: response }` → khi đó hook dùng `data.data` để lấy payload.

=> ✅ Mấu chốt: hook unwrap 1 cấp data từ baseQuery, không tự đoán key “data” từ API.

In [ ]:

1. RTK Query kỳ vọng baseQuery trả về object `{ data, error }`.
2. Khi hook như `useGetProductsQuery()` được gọi, RTK Query sẽ:
    - Nếu có `data`, trả thẳng `data` ra hook (`data` trong hook = baseQuery.data)
    - Nếu có `error`, trả `error` tương ứng.

=> Vì vậy, khi bạn dùng Axios, mà baseQuery trả về response nguyên ({ data, status, headers }), RTK Query sẽ unwrap một cấp data trong hook:

```js
// baseQuery trả về:
{
  data: { items: [...], pagination: {...} },
  status: 200,
  headers: {...}
}

// hook sẽ trả về:
const { data } = useGetProductsQuery();
console.log(data); 
// => { items: [...], pagination: {...} }   <--- RTK Query unwrap tự động

```
